In [1]:
%matplotlib inline

import os,time,sys
import tables as tb
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
import tjmonopix.analysis.interpreter_idx
import tjmonopix.analysis.interpreter_old
import pixel_clusterizer.clusterizer

COL=112
ROW=224
datdir="/sirrush/thirono/2019-01-03/tjmonopix"

In [ ]:
#import tjmonopix.scans.threshold_scan
#fname="../output_data/test.h5"
#tjmonopix.scans.threshold_scan.ThresholdScan.analyze(fname)

In [7]:
reload(tjmonopix.analysis.interpreter_idx)
subdir=os.path.join(datdir,"th")
fth_raw=os.path.join(subdir,"test.h5")
fth_hit=fth_raw[:-3]+"hit.h5"
tjmonopix.analysis.interpreter_idx.interpret_idx_h5(fth_raw,fth_hit)
tot=get_totave(fth_hit)

0 1075063 100.000% 1553603668.945s 268766hits 0errs
ndata 268766 tjonly 268766


/home/user/anaconda/envs/tid/lib/python2.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in divide
/home/user/anaconda/envs/tid/lib/python2.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


In [10]:
subdir=os.path.join(datdir,"run100")
fraw=os.path.join(subdir,"tjmonopix_100_180420-062314.h5")
fhit=os.path.join(subdir,"dut_hit.h5")
tjmonopix.analysis.interpreter_old.interpret_h5(fraw,fhit)
event_build_and_clusterize(fhit,cal_data=tot,chunck_size=10000000)

0 99999999 89.668% 3.905s 37528443hits
100000000 11522748 100.000% 4.774s 42127801hits
1.7s, 0, 9999994, 10000000 read 3.1s, convert 139.9s 9999994/42127801(23.74)
141.6s, 9999994, 19999988, 10000000 read 142.4s, convert 142.7s 19999988/42127801(47.47)
144.3s, 19999988, 29999976, 10000000 read 145.1s, convert 145.5s 29999976/42127801(71.21)
147.1s, 29999976, 39999956, 10000000 read 147.9s, convert 148.3s 39999956/42127801(94.95)
148.6s, 39999956, 42127801, 10000000 read 148.8s, convert 148.9s 42127801/42127801(100.00)


In [3]:
@njit
def _cal_tot(hits,totsum,cnt):
    #print len(hits)
    for h_i, h in enumerate(hits):
        #if h['index']!=0:
        #    print h["col"],h['row'],h['index'],totsum[h["col"],h['row'],h['index']],
        #    print float((h['te']-h['le'])&0x3F)
            #return
        cnt[h["col"],h['row'],h['index']]=cnt[h["col"],h['row'],h['index']]+1
        totsum[h["col"],h['row'],h['index']]=totsum[h["col"],h['row'],h['index']]\
                                                +float((h['te']-h['le'])&0x3F)
        #if h['index']!=0:
        #    print cnt[h["col"],h['row'],h['index']],totsum[h["col"],h['row'],h['index']],
        #    print float((h['te']-h['le'])&0x3F)
        #    return totsum,cnt
    return totsum,cnt

def get_totave(fth_hit):
    with tb.open_file(fth_hit,"a") as f:
        hits=f.root.Hits[:]
        print "ndata",len(hits),
        hits=hits[hits["col"]<COL]
        print "tjonly",len(hits)
        injlist=np.arange(35,100,1) ## TODO take from file...
        tot=np.zeros([COL,ROW,len(injlist)],dtype=np.float32)
        cnt=np.zeros([COL,ROW,len(injlist)],dtype=np.int32)
        tot,cnt=_cal_tot(hits,tot,cnt)
        tot=tot/np.array(cnt,dtype=np.float32)
        try:
            f.remove_node(f.root,"Scurve")
            f.remove_node(f.root,"ToTAve")
        except:
            pass
        f.create_carray(f.root, 
               name='Scurve', title='scurve', obj=cnt,
               filters=tb.Filters(complib='blosc', complevel=5, fletcher32=False))
        f.create_carray(f.root, 
               name='ToTAve', title='tot average', obj=tot ,
               filters=tb.Filters(complib='blosc', complevel=5, fletcher32=False))
        return tot

In [9]:
@njit
def _convert(cal_data,bins,hits,buf):
    for i,h in enumerate(hits):
        vlist=cal_data[h['col'],h['row']]
        th=float((h['te']-h['le'])&0x3F)
        tmp=np.abs(vlist-th)
        a1=np.argmin(tmp)
        #a1=0
        #a2=1
        #tpre=tmp[0]
        #for t_i,t in enumerate(tmp[1:]):
        #    if tpre > t:
        #        a1=t_i+1
        #        tpre=t
        #    else:
        #        a1=t_i
        #        break
        if a1==0:
            a2=1
        elif a1==bins.shape[0]-1:
            a2=a1-1
        elif tmp[a1+1]>tmp[a1-1]:
             a2=a1-1
        else:
             a2=a1+1
        if np.isnan(vlist[a1]) or np.isnan(vlist[a2]):
            buf[i]=vlist[a2]
        else:
            buf[i]=(bins[a2]-bins[a1])/(vlist[a2]-vlist[a1])*(vlist[a1]-th)+bins[a1]
    return buf

def event_build_and_clusterize(fhit,cal_data=None,chunck_size=10000000):

    ev_type=[("event_number","<i8"),("event_timestamp","<u8"),
             ("frame","<u1"),("column","<u1"),("row","<u1"),
             ("charge","<f4"),('flg','<u1')]
    clh_type=[('event_number', '<i8'), ('event_timestamp', '<u8'), 
             ('frame', 'u1'), ('column', 'u1'), ('row', 'u1'), ('charge', '<f4'), 
             ('flg', 'u1'), ('cluster_ID', '<i2'), 
             ('is_seed', 'u1'), ('cluster_size', '<u2'), ('n_cluster', '<u2')]
    cl_type =[('event_number', '<i8'), ('ID', '<u2'), ('n_hits', '<u2'), ('charge', '<f4'), 
              ('seed_column', '<u2'), ('seed_row', '<u2'), 
              ('mean_column', '<f4'), ('mean_row', '<f4')]

    buf=np.empty(chunck_size,dtype=ev_type)
    
    if cal_data is None:
        pass
    elif isinstance(cal_data,str):
        with tb.open_file(fcal,"r") as f:
            cal_data=f.root.ToTAve[:]
            bins=f.root.ToTAve.attrs.bins
    else:
        bins=np.arange(35,100,1)    ## this is just for debugging
        cal_data[:,:,0]=float("nan")
        
    with tb.open_file(fhit,"a") as f:
        try:
            f.remove_node(f.root,"Events")
        except:
            pass
        table_ev=f.create_table(f.root,name="Events",
                       description=np.zeros(0,dtype=ev_type).dtype,
                       title="event data")

        try:
            f.remove_node(f.root,"Clusters")
            f.remove_node(f.root,"ClusterHits")
        except:
            pass
        table_cl=f.create_table(f.root,name="Clusters",
                       description=np.zeros(0,dtype=cl_type).dtype,
                       title="clustered data")
        table_clh=f.create_table(f.root,name="ClusterHits",
                       description=np.zeros(0,dtype=clh_type).dtype,
                       title="cluster hits data")
        cl=pixel_clusterizer.clusterizer.HitClusterizer()
        cl.set_column_cluster_distance(2)
        cl.set_row_cluster_distance(2)
        cl.set_frame_cluster_distance(0)
        cl.set_hit_dtype(np.zeros(0,dtype=ev_type).dtype)

        start=0
        end=len(f.root.Hits)
        t0=time.time()
        while start<end:
            tmpend=min(start+chunck_size, end)
            hits=f.root.Hits[start:tmpend]
            if tmpend!=end:
                last_data=None
                for i in range(tmpend-start-1,-1,-1):
                    i_data=hits[i]
                    #print i,i_data
                    if last_data is None and i_data['col']<COL and i_data['cnt']==0:
                        last_data=hits[i]
                        #print "get last_data"
                    elif last_data is not None \
                        and i_data['col']<COL \
                        and last_data["timestamp"]!=i_data["timestamp"]:
                            #print "break"
                            break
                tmpend=start+i+1
                if i==0:
                    start=tmpend
                    continue
            print "%.1fs, %d, %d, %d read"%(time.time()-t0,start,tmpend, chunck_size),

            hits=hits[:tmpend-start]
            hits=hits[hits['col']<COL]
            ndata=hits.shape[0]
            if cal_data is not None:
                res=np.empty(hits.shape[0],dtype=np.float64)
                buf["charge"][:ndata] = _convert(cal_data,bins,hits,res)
            else:
                buf["charge"][:ndata]=np.array((hits['te']-hits['le'])&0x3F,dtype=np.float32)
            buf['frame'][:ndata]=hits["le"][:ndata]
            buf["event_number"][:ndata]=hits["timestamp"]
            buf["event_timestamp"][:ndata]=hits["timestamp"]
            buf['column'][:ndata]=hits["col"]
            buf['row'][:ndata]=hits["row"]
            table_ev.append(buf[:ndata])
            table_ev.flush()

            print "%.1fs, convert"%(time.time()-t0),
            buf['column']=buf['column']+1
            buf['row']=buf['row']+1
            cluster_hits,clusters=cl.cluster_hits(buf[:ndata])
            table_clh.append(cluster_hits)
            table_clh.flush()
            table_cl.append(clusters)
            table_cl.flush()

            print "%.1fs %d/%d(%.2f)"%(time.time()-t0,tmpend,end,100.0*tmpend/end)
            start=tmpend
            #break